# Data sourcing using web scraping and the Goodreads API

The purpose of this notebook is to scrape the LibraryThing website and use the Goodreads API for additional book information. The LibraryThing dataset downloaded from Prof. McAuley's <a href="https://cseweb.ucsd.edu/~jmcauley/datasets.html#social_data" target="_blank">website</a> (see data_preprocessing notebook in extra_info directory) does not contain book titles, author names or ISBSNs. In this notebook, I collect additional information for a subset of the 5000 books with the most reviews. Having the ISBNs for these books will allow me to get book genre information from the Goodreads API.

### Imports

In [15]:
import pandas as pd
from modules.book_info_extractor import extract_book_details, clean_up_dataframe, generate_clean_isbn_and_id_lists
from modules.scraper import write_htmls_to_csv
from modules.goodreads_api_functions import acquire_goodreads_id, get_book_titles, get_book_shelves
from modules.book_genre_extractor import extract_book_genre_info

### Configs

In [2]:
RAW_HTML_OUT_FOLDER_PATH = '/Users/ddefforey1/work/dsi-course/capstone_datasets'
TOP_5K_MOST_REVIEWED_BOOKS_PATH = '/Users/ddefforey1/work/dsi-course/capstone_datasets/top_5K_books.csv'
CLEAN_BOOK_GENRES_PATH = '/Users/ddefforey1/work/dsi-course/capstone_datasets/clean_book_genres.csv'

### Web Requests and Writing Output to a CSV

In [3]:
# importing the book ids for 5K books with the most reviews
top_5K_books = pd.read_csv(TOP_5K_MOST_REVIEWED_BOOKS_PATH)

In [4]:
# make a list of book IDs
books_list = list(top_5K_books.book_id)

In [5]:
raw_htmls_path = write_htmls_to_csv(books_list=books_list[:5], path=RAW_HTML_OUT_FOLDER_PATH)


Pages scraped incorrectly: []


### Parsing Data with BeautifulSoup

In [6]:
# loading the csv containing the raw htmls
raw_htmls_path = '/Users/ddefforey1/work/dsi-course/capstone_datasets/scraped_raw_html_librarything_19-05-02T13:40:13.csv'
raw_data = pd.read_csv(raw_htmls_path)
raw_data.head()

,book_id,raw_html
0,4979986,<!DOCTYPE html><html>\n<head><title>The Hunger...
1,8384326,<!DOCTYPE html><html>\n<head><title>Twilight b...
2,1541442,<!DOCTYPE html><html>\n<head><title>The Girl w...
3,393681,<!DOCTYPE html><html>\n<head><title>The Book T...
4,8662515,<!DOCTYPE html><html>\n<head><title>Catching F...


In [7]:
book_info = raw_data.raw_html.apply(extract_book_details).copy()

In [8]:
book_info = clean_up_dataframe(df=book_info.copy(), books_list=books_list)
book_info.head()

,id,book_title,author,isbn
0,4979986,The Hunger Games,Suzanne Collins,0439023483
1,8384326,Twilight (2005),Stephenie Meyer,0316015849
2,1541442,The Girl with the Dragon Tattoo (2005),Stieg Larsson,0307454541
3,393681,The Book Thief (2007),Markus Zusak,0375842209
4,8662515,Catching Fire,Suzanne Collins,0439023491


In [9]:
book_info.shape

(3638, 4)

Some books are missing ISBNs because this information was not included on their webpage on the LibraryThing website. At this point, I will continue with the ones that have ISBNs but it is worth noting that this project could be expanded by including books with missing ISBNs.

### Collect Goodreads Book Identifiers using Goodreads API

Having Goodreads book IDs will allow us to collect book genre information, the last piece of information needed for the topic model.

In [10]:
isbns_list, id_list = generate_clean_isbn_and_id_lists(df=book_info.copy())

In [18]:
goodreads_ids = acquire_goodreads_id(isbns_list[:5])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# making a dataframe with ISBNs and their corresponding Goodreads IDs
goodreads_book_details = pd.DataFrame({
    'id': id_list[:5],
    'isbn': isbns_list[:5],
    'goodreads_id': goodreads_ids
})

### Collect book genre information using Python interface for Goodreads

In [ ]:
# collect book titles from Goodreads to confirm that they match those from the LibraryThing dataset
goodreads_book_details['goodreads_book_titles'] = get_book_titles(goodreads_book_details.goodreads_id[:5])

In [ ]:
# collect goodreads shelves (from which genre info will be derived)
goodreads_book_details['goodreads_shelves'] = get_book_shelves(goodreads_book_details.goodreads_id)

In [ ]:
goodreads_book_details.head()

### Extract book genre information

In [ ]:
# removing an academic textbook
goodreads_book_details = pd.read_csv('/Users/ddefforey1/work/dsi-course/capstone_datasets/raw_goodreads_ids_genres.csv')
goodreads_book_details = goodreads_book_details.drop(index=566, axis=0)
goodreads_book_details = goodreads_book_details.reset_index(drop=True)

In [ ]:
# extract book genres from goodreads shelves
goodreads_book_details['book_genres'] = goodreads_book_details.goodreads_shelves.map(extract_book_genre_info)
goodreads_book_details.head()

In [ ]:
# saving the dataframe for future use
#goodreads_book_details.to_csv(CLEAN_BOOK_GENRES_PATH, index=False)